<a href="https://colab.research.google.com/github/GabrielBatavia/Learn_TensorFlow/blob/main/NLP_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sqeuences text or speech).

Another common term for NLP problems is sequence to sequence problems

## Check for GPU

In [ ]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


## Get helper functions

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-05-24 07:45:24--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-05-24 07:45:24 (66.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster)



In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-05-24 07:45:31--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.98.207, 142.250.97.207, 142.251.107.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2024-05-24 07:45:31 (93.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualize a text dataset

In [ ]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
train_df_shuffled["text"][2227]

'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@'

In [ ]:
# What does the test dataframe look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# How many examples of each class
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [ ]:
# How many total sample
len(train_df), len(test_df)

(7613, 3263)

In [ ]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)# create random indexes
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"Target: {target}", "(real diaster)" if target > 0 else "(not real diaster)")
  print(f"Text:\n{text}\n")
  print("==-\n")


Target: 1 (real diaster)
Text:
Dramatic Video Shows Plane Landing During Violent Storm http://t.co/rJ9gkJKJJn

==-

Target: 0 (not real diaster)
Text:
forever drowning in my feelings.

==-

Target: 1 (real diaster)
Text:
Bamenda Floods Kill Animals Birds - http://t.co/WnamtxlfMt http://t.co/6cOIDv11qV

==-

Target: 1 (real diaster)
Text:
Turkish troops killed in Kurdish militant 'suicide attack' http://t.co/wD7s6S0vci

==-

Target: 0 (not real diaster)
Text:
FedEx no longer shipping bioterror germs - WXIA-TV | @scoopit http://t.co/ZQqJrQsbJm

==-



### Split data into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Use train_test_split to split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data to validation
                                                                            random_state=42)

In [ ]:
# check len data
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
# check some example
train_sentences[:5], train_labels[:5]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
       dtype=object),
 array([0, 0, 1, 0, 0]))

## Convert text into numbers

Tokenization vs embeddings

When dealing with a text problem, one of the first things we cant do before we build a model is convers our text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding = create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [ ]:
 # Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=10000, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words
                                    output_mode="int",
                                    output_sequence_length = None, # if its None its automaticly use the max long length
                                    pad_to_max_tokens = True)

In [ ]:
train_sentences[0].split(), len(train_sentences[0].split())

(['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet'], 7)

In [ ]:
# Find the average number of tokens in the training tweets
round(sum([len(i.split()) for i in train_sentences])) / len(train_sentences)

14.901036345059115

In [ ]:
# Setup text vectorization variables
max_vocab_length = 10000  # max number of words to have in our vocabulary
max_length = 15  # max length our sequences will be (r.g. how many words from a Tweet foes a model see)

# recreate and spesific the value
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_sequence_length = max_length)

In [ ]:
# Fit the text vectorizer to training text
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a blood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 353,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text : \n {random_sentence} \n Vectorized version :")
text_vectorizer([random_sentence])


Original text : 
 Officials say a quarantine is in place at a Birmingham home over a possible Ebola case.

Edward Khan of the... http://t.co/qnIJ2p8Zv6 
 Vectorized version :


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 503,  132,    3,  417,    9,    4,  596,   17,    3, 6099,  153,
          60,    3,  533, 1085]])>

In [ ]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all unique words
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Number 5 most common words :\n {top_5_words}")
print(f"Number 5 least common words :\n {bottom_5_words}")

Number of words in vocab: 10000
Number 5 most common words :
 ['', '[UNK]', 'the', 'a', 'in']
Number 5 least common words :
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding layer

To make our embedding, we're going to use tensorflow embedding layers

The parameters we care most about for our embedding layer:
* input_dim = the size of our vocabulary
* output_dim = the size of the output embedding vector, for example : a value of 100 would mean each token gets represented by a vector 100 long
* input_length = length of  the sequences being passed to the embedding layers

In [ ]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length, # set input shape
                             output_dim = 128, # output shape
                             input_length = max_length) # how long is each input

embedding

In [ ]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text : \n {random_sentence} \n Embedded version :")

# Embed the random sentence (turn it into dense of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text : 
 Oops.
H bomb lost 70 miles off the Okinawan coast.
Fell off the ship 1965.
http://t.co/yVsJyzwxJR 
 Embedded version :


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00735425,  0.02402769,  0.02568248, ..., -0.02097004,
          0.0044601 , -0.0471367 ],
        [ 0.01608069,  0.02095331,  0.01053282, ...,  0.02242282,
          0.01219468, -0.04878755],
        [-0.03450298, -0.02483267,  0.01353553, ...,  0.02487561,
          0.01502134, -0.02586891],
        ...,
        [ 0.00059782, -0.01713466,  0.04504386, ...,  0.03546042,
          0.02180337, -0.03218259],
        [ 0.00623306,  0.04996241,  0.00243914, ...,  0.00577282,
          0.01058897, -0.02185508],
        [-0.00978366, -0.01652617,  0.01888363, ..., -0.00445117,
         -0.03441941,  0.00761682]]], dtype=float32)>

In [ ]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00735425,  0.02402769,  0.02568248, -0.00801046,  0.03454522,
        -0.00826592,  0.00818228,  0.04246367, -0.0223212 ,  0.03420024,
        -0.02718362, -0.02335621,  0.02099316, -0.01288199, -0.01904571,
         0.04452931,  0.01877042, -0.03614017, -0.01954748,  0.03127012,
        -0.04621438, -0.01309689, -0.01581942,  0.01304122, -0.02891386,
        -0.04929885, -0.03861935, -0.04302695, -0.04456091,  0.03575654,
         0.03995224, -0.01831385, -0.03854003,  0.04465986, -0.00610941,
        -0.01722239, -0.00677601,  0.01249733,  0.00420265, -0.0046676 ,
        -0.03679727, -0.04654954,  0.03868895, -0.03412928,  0.03748521,
        -0.03047231,  0.0482781 ,  0.02867745, -0.04029506, -0.04586638,
        -0.01661581, -0.00684823,  0.03723891, -0.04134549, -0.03134471,
         0.0352734 ,  0.0402101 , -0.01687851, -0.00822593,  0.04362316,
         0.01250217, -0.02328944, -0.0366276 , -0.02647171, -0.00797638,
  

## Modelling a text dataset (running a series of experiments)

we'll start with a baseline and move on from there

* Model 0 : Naive bayes (baseline)
* Model 1 : Feed_forward neural netwrok (dense model)
* Model 2 : LSTM model (RNN)
* Model 3 : GPU model (RNN)
* Model 4 : Bidirectional-LSTM model (RNN)
* Model 5 : 1D Convolutional Neural Network (CNN)
* Model 6 : TensorFlow Hub Pretrained Feature Extractor (using transder learning for NLP)
* Model 7 : Same as model 6 with 10% of training data


### Model 0 : Getting a baseline

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modeling pipeline

model_0 = Pipeline(
    [
        ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
        ("clf", MultinomialNB()) # model the text
    ]
)

# Fit the pipleine to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
# Evaluate our model 0
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score * 100:2f}%")

Our baseline model achieves an accuracy of: 79.265092%


In [ ]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [ ]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however, this will be cumbersome and could

Lets create one to compare our model's predictions with the truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score

for a deep overview of many different evaluation methods, see this : https://scikit-learn.org/stable/modules/model_evaluation.html

In [ ]:
# Funtion to evaluate : accurary, precision, recall, f1-score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1-score using "wighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {
      "accuracy": model_accuracy,
      "precision": model_precision,
      "recall": model_recall,
      "f1": model_f1
  }

  return model_results

In [ ]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels, y_pred=baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1 : A Simple model

In [34]:
# Create a tensorboard callback (need to create a new one for eacg model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [35]:
# Build model with the Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs

outputs = layers.Dense(1, activation="sigmoid")(x) # Create the output layer, use sigmoid cause we want binary output
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")


In [37]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 dense (Dense)               (None, 15, 1)             129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 1280129 (4.88 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [42]:
import numpy as np

# Convert labels to numpy arrays if they are not already
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

# Print the shapes of the labels
print("Shape of train_labels:", train_labels.shape)
print("Shape of val_labels:", val_labels.shape)

# Reshape labels to have a shape of (number_of_samples, 1)
train_labels = train_labels.reshape(-1, 1)
val_labels = val_labels.reshape(-1, 1)

# Print the shapes again to confirm
print("Shape of reshaped train_labels:", train_labels.shape)
print("Shape of reshaped val_labels:", val_labels.shape)

Shape of train_labels: (6851,)
Shape of val_labels: (762,)
Shape of reshaped train_labels: (6851, 1)
Shape of reshaped val_labels: (762, 1)


In [43]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                         experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240524-084903
Epoch 1/5


ValueError: Creating variables on a non-first call to a function decorated with tf.function.